## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Longyearbyen,SJ,78.2186,15.6401,6.64,66,75,9.22,broken clouds
1,1,Salalah,OM,17.0151,54.0924,73.49,35,0,6.91,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,76.44,73,0,17.34,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.91,50,0,13.80,clear sky
4,4,Upernavik,GL,72.7868,-56.1549,26.78,96,100,9.44,snow


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,76.44,73,0,17.34,clear sky
5,5,Hobart,AU,-42.8794,147.3294,87.03,57,20,1.01,few clouds
11,11,Vaini,TO,-21.2000,-175.2000,86.16,79,20,11.50,few clouds
18,18,Atuona,PF,-9.8000,-139.0333,78.58,79,57,21.27,light rain
26,26,Charters Towers,AU,-20.1000,146.2667,88.32,39,1,6.78,clear sky
34,34,Catanduva,BR,-21.1378,-48.9728,76.87,53,8,1.14,clear sky
35,35,Sibu,MY,2.3000,111.8167,77.05,89,76,2.01,broken clouds
38,38,Avarua,CK,-21.2078,-159.7750,82.45,78,20,16.11,light rain
49,49,Pacific Grove,US,36.6177,-121.9166,78.91,37,1,0.00,clear sky
52,52,Guarapari,BR,-20.6667,-40.4975,76.89,95,17,5.35,few clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()  #There are no empty rows

City_ID                192
City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy() 
clean_df.dropna(inplace = True)   # Though there are no empty rows but still used the method to demonstrate the use
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,76.44,clear sky,-23.1203,-134.9692,
5,Hobart,AU,87.03,few clouds,-42.8794,147.3294,
11,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,
18,Atuona,PF,78.58,light rain,-9.8000,-139.0333,
26,Charters Towers,AU,88.32,clear sky,-20.1000,146.2667,
34,Catanduva,BR,76.87,clear sky,-21.1378,-48.9728,
35,Sibu,MY,77.05,broken clouds,2.3000,111.8167,
38,Avarua,CK,82.45,light rain,-21.2078,-159.7750,
49,Pacific Grove,US,78.91,clear sky,36.6177,-121.9166,
52,Guarapari,BR,76.89,few clouds,-20.6667,-40.4975,


In [36]:
# To save the skipped value as NaN in the data frame
import numpy as np
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.dropna(inplace = True)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,76.44,clear sky,-23.1203,-134.9692,People ThankYou
5,Hobart,AU,87.03,few clouds,-42.8794,147.3294,Mantra on Collins Hobart
11,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Atuona,PF,78.58,light rain,-9.8000,-139.0333,Villa Enata
26,Charters Towers,AU,88.32,clear sky,-20.1000,146.2667,Cattleman's Rest Motor Inn


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))